# Projeto 1 - Ciência dos Dados

Nome: Enrico Lopez Borges de Carvalho

Nome: Luiza Valezim Augusto Pinto

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline

import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import os

from IPython.display import display

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
filename = 'Intel3.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Intel3.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train_raw = pd.read_excel(filename)
train_raw.head(5)

,Treinamento,Relevância
0,@culper_intel @mikayesfiona culper is correct,0
1,notebook 2 em 1 dell inspiron i14-5481-m11 8ª ...,0
2,"meu pc quase queimou, ligava ele, abria o linu...",1
3,é aqui o funeral da amd? e a intel acomodada q...,2
4,ferramentas linux: os patches do driver de grá...,1


In [4]:
test_raw = pd.read_excel(filename, sheet_name = 'Teste')
test_raw.head(5)

,Teste,Relevância
0,@victrixhatii tava vendo o ryzen ontem. cabulo...,1
1,🔈 🚨 alerta 1: renda brasil morre no ninho : bo...,0
2,"@fujiwaraca eu vi, agora as coisas vão ficar i...",2
3,rt @stabeto: secretário de estado dos eua fará...,0
4,rt @vrt_ramp: pra galera que quer comprar um p...,1


___
## Classificador automático de sentimento


***Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.***

O produto escolhido foi a marca Intel, pois, desde que a Nvidea (concorrente da Intel) comprou um novo processador para competir no mercado, houve várias notícias discutindo a disputa. Assim, consideramos relevantes os tweets que mais agregavam para essa discussão, além, também, daqueles que falavam bem ou mal da marca com certo nível de argumentação.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
train_raw['Treinamento'].astype('str')
test_raw['Teste'].astype('str')
test_raw.dtypes

Teste         object
Relevância     int64
dtype: object

In [6]:
train_raw['Relevância'] = train_raw['Relevância'].astype('category')
train_raw.Relevância.cat.categories = ['Irrelevante', "Meio Relevante",'Relevante']
train_raw.head()


test_raw['Relevância'] = test_raw['Relevância'].astype('category')
test_raw.Relevância.cat.categories = ['Irrelevante',"Meio Relevante","Relevante"]
test_raw.head()

,Teste,Relevância
0,@victrixhatii tava vendo o ryzen ontem. cabulo...,Meio Relevante
1,🔈 🚨 alerta 1: renda brasil morre no ninho : bo...,Irrelevante
2,"@fujiwaraca eu vi, agora as coisas vão ficar i...",Relevante
3,rt @stabeto: secretário de estado dos eua fará...,Irrelevante
4,rt @vrt_ramp: pra galera que quer comprar um p...,Meio Relevante


In [7]:
# CLASSIFICAÇÃO DE RELEÂNCIA NO DATAFRAME'TRAIN'

train_irrelevantes = train_raw.loc[train_raw['Relevância']=='Irrelevante',:]
train_meio_relevantes = train_raw.loc[train_raw['Relevância'] =="Meio Relevante",:]
train_relevantes = train_raw.loc[train_raw['Relevância']=="Relevante",:]


In [8]:
# JUNÇÃO DE TODAS OS TWEETS DO DATAFRAME 'TRAIN'

treinamento_textao_relevantes = " ".join(train_relevantes.Treinamento)
treinamento_textao_irrelevantes = " ".join(train_irrelevantes.Treinamento)
treinamento_textao_meio_relevantes = " ".join(train_meio_relevantes.Treinamento)

treinamento_textao = ' '.join(train_raw.Treinamento)

In [9]:
# LIMPEZA DOS CARACTÉRES NO DATAFRAME 'TRAIN'

import re 
def cleanup(text):
    punctuation = '[\n!-.,:;?|/"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed1 = re.sub(r'http\S+','',text_subbed)
    text_subbed2 = re.sub(r'@\S+',"",text_subbed1)
    text_subbed3 = re.sub('   ', ' ',text_subbed2)
    text_subbed4 = re.sub('  ', ' ',text_subbed3)
    return text_subbed4

train_limpo = cleanup(treinamento_textao.lower())
train_relevantes_limpo = cleanup(treinamento_textao_relevantes.lower())
train_irrelevantes_limpo = cleanup(treinamento_textao_irrelevantes.lower())
train_meio_relevantes_limpo = cleanup(treinamento_textao_meio_relevantes.lower())


In [10]:
# SEPARAÇÃO DAS PALAVRAS DO DATAFRAME 'TRAIN'

todas_palavras_train = train_limpo.split()
todas_palavras_train_relevantes = train_relevantes_limpo.split()
todas_palavras_train_irrelevantes = train_irrelevantes_limpo.split()
todas_palavras_train_meio_relevantes = train_meio_relevantes_limpo.split()


In [11]:
# TABELA DAS PALAVRAS DO DATAFRAME 'TRAIN'

serie_treinamento = pd.Series(todas_palavras_train)
serie_treinamento_relevantes = pd.Series(todas_palavras_train_relevantes)
serie_treinamento_irrelevantes = pd.Series(todas_palavras_train_irrelevantes)
serie_treinamento_meio_relevantes = pd.Series(todas_palavras_train_meio_relevantes)


In [12]:
# TABELA RELATIVA DAS PALAVRAS DO DATAFRAME 'TRAIN'

tabela_treinamento_quant = serie_treinamento.value_counts()
tabela_treinamento_relevantes_quant= serie_treinamento_relevantes.value_counts()
tabela_treinamento_irrelevantes_quant = serie_treinamento_irrelevantes.value_counts()
tabela_treinamento_meio_relevantes_quant= serie_treinamento_meio_relevantes.value_counts()


In [13]:
tabela_treinamento_relevantes_quant.head()

a        79
intel    57
e        48
que      41
amd      36
dtype: int64

In [14]:
tabela_treinamento_irrelevantes_quant.head()

de       125
intel    118
um        62
e         60
o         45
dtype: int64

In [15]:
tabela_treinamento_meio_relevantes_quant.head()

intel    72
de       59
e        46
a        44
um       39
dtype: int64

In [16]:
# DEFINIÇÃO DA PROBABILIDADE DE RELEVÂNCIA A PRIORI

P_r = train_relevantes.size/train_raw.size
P_ir = train_irrelevantes.size/train_raw.size
P_mr = train_meio_relevantes.size/train_raw.size

print(f'Probabilidade a priori de não ser relevante: {P_ir:0.5g}')
print(f'Probabilidade a priori de ser meio relevante: {P_mr:0.5g}')
print(f'Probabilidade a priori de ser relevante: {P_r:0.5g}')


Probabilidade a priori de não ser relevante: 0.56333
Probabilidade a priori de ser meio relevante: 0.23333
Probabilidade a priori de ser relevante: 0.20333


In [17]:
frase = "amd é"

def verifica_relevancia(frase):
    frase = cleanup(frase.lower())
    frase = frase.split()
    Pi_frase = 1
    P_frase = 1
    P_meio_frase=1

    for el in frase:
        if el not in todas_palavras_train_relevantes:
            valor_rele=0
        else:
            valor_rele=tabela_treinamento_relevantes_quant[el]

        if el not in todas_palavras_train_irrelevantes:
            valor_irrele=0
        else:
            valor_irrele=tabela_treinamento_irrelevantes_quant[el]

        if el not in todas_palavras_train_meio_relevantes:
            valor_mei_rele=0
        else:
            valor_mei_rele=tabela_treinamento_meio_relevantes_quant[el]

        P_frase*=(valor_rele+1)/(tabela_treinamento_relevantes_quant.size + tabela_treinamento_quant.size)
        Pi_frase*=(valor_irrele+1)/(tabela_treinamento_irrelevantes_quant.size + tabela_treinamento_quant.size)
        P_meio_frase*=(valor_mei_rele+1)/(tabela_treinamento_meio_relevantes_quant.size + tabela_treinamento_quant.size)

    P_ir_frase = Pi_frase * P_ir
    P_mr_frase = P_meio_frase * P_mr
    P_r_frase = P_frase * P_r

    if P_ir_frase > P_mr_frase and P_ir_frase > P_r_frase:
        return(0)
    elif P_mr_frase > P_ir_frase and P_mr_frase > P_r_frase:
        return(1)
    elif P_r_frase > P_ir_frase and P_r_frase > P_mr_frase:
        return(2)

    
print(verifica_relevancia(frase))

2


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [18]:
lista_relevancia_teste=[]
for ele in test_raw["Teste"]:
    i=verifica_relevancia(ele)
    lista_relevancia_teste.append(i)
    
test_raw["Relevancia_test"] = lista_relevancia_teste

In [19]:
test_raw['Relevancia_test'].astype('str')
test_raw['Relevancia_test'] = test_raw['Relevancia_test'].astype('category')
test_raw['Relevancia_test'].cat.categories = ['Irrelevante',"Meio Relevante","Relevante"]

In [20]:
test_raw.head(5)

,Teste,Relevância,Relevancia_test
0,@victrixhatii tava vendo o ryzen ontem. cabulo...,Meio Relevante,Irrelevante
1,🔈 🚨 alerta 1: renda brasil morre no ninho : bo...,Irrelevante,Irrelevante
2,"@fujiwaraca eu vi, agora as coisas vão ficar i...",Relevante,Relevante
3,rt @stabeto: secretário de estado dos eua fará...,Irrelevante,Irrelevante
4,rt @vrt_ramp: pra galera que quer comprar um p...,Meio Relevante,Irrelevante


In [21]:
ttes_irrelevantes = test_raw.loc[test_raw['Relevância']=='Irrelevante',:]
ttes_meio_relevantes = test_raw.loc[test_raw['Relevância'] =="Meio Relevante",:]
ttes_relevantes = test_raw.loc[test_raw['Relevância']=="Relevante",:]

In [22]:
P_tr = ttes_relevantes.size/test_raw.size
P_tir = ttes_irrelevantes.size/test_raw.size
P_tmr = ttes_meio_relevantes.size/test_raw.size

print(f'Probabilidade a priori de não ser relevante: {P_tir:0.5g}')
print(f'Probabilidade a priori de ser meio relevante: {P_tmr:0.5g}')
print(f'Probabilidade a priori de ser relevante: {P_tr:0.5g}')

Probabilidade a priori de não ser relevante: 0.68421
Probabilidade a priori de ser meio relevante: 0.15789
Probabilidade a priori de ser relevante: 0.15789


In [25]:
# crostab
pd.crosstab(test_raw.Relevância, test_raw.Relevancia_test,normalize = 'columns').round(4) *100

Relevancia_test,Irrelevante,Meio Relevante,Relevante
Relevância,,,
Irrelevante,84.52,78.95,36.73
Meio Relevante,14.29,15.79,18.37
Relevante,1.19,5.26,44.90


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis ✅
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis ✅
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação ✅
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B) ✅
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento ✅
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto ✅
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa) ✅
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados // 
___

***Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a
performance do seu classificador.***

***R:*** Nosso classificador funciona bem para tweets não relevantes , mas peca para meio relevantes e os relevantes. Isso ocorre por conta da base de tweets extraídos, nela a base train contem em maioria tweets  não relevantes o que favorece a probabilidade do conteúdo já ser considerado não relevante . Mas o que realmente atrapalha o classificador é que a maioria desses tweets não relevantes são anúncios que geralmente contem muitas palavras , desse modo uma palavra que deveria ser "relevante" por exemplo aparece muitas vezes nesses anúncios atrapalhando a probabilidade das palavras serem classificadas corretamente.

***Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?***

***R:*** O plano de expansão proposto baseia-se na obtenção maior de dados relevantes dos usuários. Isso porque, quanto maior for a base de dados obtida, maior o aprimoramento do classificador em obter uma porcentagem de acerto mais precisa. Essa diferenciação das demais empresas concorrentes será vital para as futuras tomadas de decisão de nossa empresa, já que a opinião e gosto do público crítico será levada em consideração, aumentando as probabilidades de lucro futuramente. Além disso, os amantes do produto podem trazer ideias construtivas e inovadoras que, posteriormente, poderão ser implementadas, solucionando as dores dos usuários. Com a Era da Internet as informações circulam quase que no mesmo instante, dessa forma é essencial manter nossa imagem positiva e mostrar que somos próximos de nossos consumidores e atenciosos à suas reclamações, assim, formando uma comunidade fortificada a qual que não nos abandonará em momentos de crise. Para isso, sugerimos a análise de dados mais aprofundada no aplicativo Twitter, como também em outras redes sociais (como Instagram), em grupos de Facebook voltados para tecnologia, fóruns de discussão, etc... lugares, esses, de fácil acesso ao nosso público-alvo. Porém, para isso, precisamos de uma continuação do financiamento do projeto para conseguirmos seguir com nosso objetivo de interação e fazer nossa diferença em nossa área de atuação.

***Explique como são tratadas as mensagens com dupla negação e sarcasmo.***

***R:*** Tweets com dupla negação podem ter sido (e provavelmente foram) classificados erroneamente. Tal erro pode ser explicado com base no método que estamos os classificando, pois, o nosso computador não sabe que a dupla negação é a mesma coisa que uma confirmação. Dessa forma, ele só contaria as palavras "não", as quais, provavelmente, teriam uma probabilidade maior ou de irrelevantes ou relevantes, o enviesando e não necessariamente considerando seu real significado. A mesma coisa ocorre com o uso do sarcasmo. O nosso classificador não entende quando uma pessoa está sendo sarcástica, de modo que a chance dele errar na classificação é grande.


***Por que não podemos utilizar o próprio classificador para gerar mais amostras de treinamento?***

***R:*** Não podemos por dois motivos. O primeiro é de que nossa base de dados é muito pequena ainda para o computador entender coesão e coerência de frases, ou seja, ele não saberia escrever frases que façam sentido. O segundo motivo seria de que, a partir do momento que ele cria sua própria base de treinamento, ele não estaria mais aprendendo e sim replicando o que já conhece, portanto, por mais que ele escrevesse mais infinitos exemplos, ele continuaria escrevendo frases sem sentido, pois, para ele, ele está certo.

***Propor diferentes cenários para Naive Bayes fora do contexto do projeto***

***R:*** Cenários:

- Já vi um youtuber utilizar um código baseado em Naive Bayes para classificar quais comentários dos vídeos do Youtube dele seriam relevantes. Ele fez isso para encontrar aqueles comentários que realmente falavam sobre o vídeo e propuseram críticas construtivas para ele melhorar. Link: https://www.youtube.com/watch?v=MDtdJBqrIi8&ab_channel=JohnFish.
- Igual ao exemplo dado no handout, podemos utilizar o código para diferenciar coisas como: idioma de artigos/livros, palavras frequentes de autores/cantores, podemos também pegar cifras de todas as músicas de uma banda e tentar encontrar os acordes e as palavras mais comuns.
- Ao tentar fazer uma Inteligência Artificial para um chat de suporte, por exemplo, precisamos treiná-la para entender como se comunicar com o público-alvo. Dessa forma, uma maneira de fazer isso é, caso o público-alvo seja jovem, pegar amostras de tweets, posts no Instagram e Facebook, etc, e rodar um código Naive Bayes para criar uma base de dados real e condizente com o público. Basicamente, o método Naive Bayes é passível de ser usado na maioria dos casos de Machine Learning, dependendo de seu intuito.

***Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)***

***R:*** Para melhorarmos nosso código, podemos:

- Coletar uma base de dados de treinamento maior. Para isso, podemos buscar em mais lugares além do Twitter (como Instagram, Facebook, blogs, etc...) já que, dependendo do tema, o número de tweets é reduzido. Podemos coletar esses dados em outras línguas e depois fazendo algum sistema de tradução.
- Diminuir o número de palavras neutras. O que quero dizer com isso é retirar do nosso código palavras que podem ser usadas em contextos tanto relevantes quanto irrelevantes (como 'vc', 'ele', 'eh',etc...), pois elas podem mais confundir o computador do que nos ajudar. Não é porque na nossa amostra, por exemplo, as frases que consideramos como irrelevantes tem mais palavras como "vc" que todos os tweets que a tem serão, por definição, irrelevantes. Ao mesmo tempo, não podemos ignorar o fato de que, caso o comentário tenha sido escrito com português sem girias e abreviações, a possibilidade de ser relevante possa ser significativamente maior. Link de um artigo que comenta sobre isso: https://www.researchgate.net/publication/302979046_The_Importance_of_Neutral_Class_in_Sentiment_Analysis_of_Arabic_Tweets
- Considerar mais (ou dar maior peso) para aqueles tweets/comentários que apresentam maior repercussão. Por exemplo, aumentando a relevância de um comentário que tenha mais likes e/ou mais retweets. O que poderia ser feito também é considerar a relevância da pessoa na plataforma (Exemplo: Uma pessoa de 10 seguidores em comparação com uma de 10 milhões), mas por motivos de que isso já afeta o número de likes ou retweets talvez não seja algo muito necessário.
- Talvez utilizar outros meios além do classificador de Naive Bayes possa ser complementar, como o Vader, o qual já tem sua base de dados definida com palavras já categorizadas (em inglês) com seu nível de positividade. Dessa forma, teríamos já uma base de treinamento para nosso programa mais forte que só alguns tweets e mais consistente, pois não só duas pessoas classificaram se é relevante ou irrelevante. Link para um artigo falando mais do VADER: https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664 // Link de um GitHub que usa esse método: https://github.com/johnafish/senticomment
- Outra forma seria também de coletar dados sobre outros assuntos também para comparação. Isso por quê um grupo que comenta mais sobre determinado assunto pode não representar para o analísta/ para a impresa pontos muito importantes. Portanto uma forma de verificar o nível de relevância que está sendo avaliado pode mudar dependendo do público-alvo.
- Outra forma de melhorar o código seria criar outro classificador para verificar se um tweet é um anuncio ou nao e se este fosse classificado como True o conteúdo dele ou seja as palavras nao entrariam para a probabilidade delas serem relevantes ou nao e sim so para a probabilidade a prior dele ser relevante ou não. 

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**